# VertexAIEmbeddings

[Google Vertex](https://cloud.google.com/vertex-ai) is a service that exposes all foundation models available in Google Cloud.

This will help you get started with Google Vertex AI [embedding models](/docs/concepts/embedding_models) using LangChain. For detailed documentation on `VertexAIEmbeddings` features and configuration options, please refer to the [API reference](https://api.js.langchain.com/classes/langchain_google_vertexai.GoogleVertexAIEmbeddings.html).

## Overview
### Integration details

| Class | Package | Local | [Py support](https://python.langchain.com/docs/integrations/text_embedding/google_vertex_ai_palm/) | Package downloads | Package latest |
| :--- | :--- | :---: | :---: |  :---: | :---: |
| [`VertexAIEmbeddings`](https://api.js.langchain.com/classes/langchain_google_vertexai.GoogleVertexAIEmbeddings.html) | [`@langchain/google-vertexai`](https://npmjs.com/@langchain/google-vertexai) | ❌ | ✅ | ![NPM - Downloads](https://img.shields.io/npm/dm/@langchain/google-vertexai?style=flat-square&label=%20&) | ![NPM - Version](https://img.shields.io/npm/v/@langchain/google-vertexai?style=flat-square&label=%20&) |

## Setup

LangChain.js supports two different authentication methods based on whether
you're running in a Node.js environment or a web environment.

To access `ChatVertexAI` models you'll need to setup Google VertexAI in your Google Cloud Platform (GCP) account, save the credentials file, and install the `@langchain/google-vertexai` integration package.

### Credentials

Head to your [GCP account](https://console.cloud.google.com/) and generate a credentials file. Once you've done this set the `GOOGLE_APPLICATION_CREDENTIALS` environment variable:

```bash
export GOOGLE_APPLICATION_CREDENTIALS="path/to/your/credentials.json"
```

If running in a web environment, you should set the `GOOGLE_VERTEX_AI_WEB_CREDENTIALS` environment variable as a JSON stringified object, and install the `@langchain/google-vertexai-web` package:

```bash
GOOGLE_VERTEX_AI_WEB_CREDENTIALS={"type":"service_account","project_id":"YOUR_PROJECT-12345",...}
```

If you want to get automated tracing of your model calls you can also set your [LangSmith](https://docs.smith.langchain.com/) API key by uncommenting below:

```bash
# export LANGCHAIN_TRACING_V2="true"
# export LANGCHAIN_API_KEY="your-api-key"
```

### Installation

The LangChain `VertexAIEmbeddings` integration lives in the `@langchain/google-vertexai` package:

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/google-vertexai @langchain/core
</Npm2Yarn>
```

## Instantiation

Now we can instantiate our model object and embed text:

In [1]:
import { VertexAIEmbeddings } from "@langchain/google-vertexai";
// Uncomment the following line if you're running in a web environment:
// import { VertexAIEmbeddings } from "@langchain/google-vertexai-web"

const embeddings = new VertexAIEmbeddings({
  model: "text-embedding-004",
  // ...
});

## Indexing and Retrieval

Embedding models are often used in retrieval-augmented generation (RAG) flows, both as part of indexing data as well as later retrieving it. For more detailed instructions, please see our RAG tutorials under the [working with external knowledge tutorials](/docs/tutorials/#working-with-external-knowledge).

Below, see how to index and retrieve data using the `embeddings` object we initialized above. In this example, we will index and retrieve a sample document using the demo [`MemoryVectorStore`](/docs/integrations/vectorstores/memory).

In [2]:
// Create a vector store with a sample text
import { MemoryVectorStore } from "langchain/vectorstores/memory";

const text = "LangChain is the framework for building context-aware reasoning applications";

const vectorstore = await MemoryVectorStore.fromDocuments(
  [{ pageContent: text, metadata: {} }],
  embeddings,
);

// Use the vector store as a retriever that returns a single document
const retriever = vectorstore.asRetriever(1);

// Retrieve the most similar text
const retrievedDocuments = await retriever.invoke("What is LangChain?");

retrievedDocuments[0].pageContent;

LangChain is the framework for building context-aware reasoning applications


## Direct Usage

Under the hood, the vectorstore and retriever implementations are calling `embeddings.embedDocument(...)` and `embeddings.embedQuery(...)` to create embeddings for the text(s) used in `fromDocuments` and the retriever's `invoke` operations, respectively.

You can directly call these methods to get embeddings for your own use cases.

### Embed single texts

You can embed queries for search with `embedQuery`. This generates a vector representation specific to the query:

In [3]:
const singleVector = await embeddings.embedQuery(text);

console.log(singleVector.slice(0, 100));

[
    -0.02831101417541504,   0.022063178941607475,  -0.07454229146242142,
    0.006448323838412762,   0.001955120824277401, -0.017617391422390938,
       0.018649872392416,   -0.05262855067849159, 0.0006953597767278552,
  -0.0018249079585075378,   0.022437218576669693, 0.0036489504855126143,
   0.0018086736090481281,   0.016940006986260414, -0.007894322276115417,
    -0.04187627509236336,   0.039501357823610306,   0.06918870657682419,
   -0.006931832991540432,   0.049655742943286896,  0.021211417391896248,
   -0.029322246089577675,   -0.04546992480754852,  -0.01769082061946392,
    0.046703994274139404,    0.03127637133002281,  0.006355373188853264,
    0.014901148155331612,  -0.006893016863614321,  -0.05992589890956879,
   -0.009733330458402634,   0.015709295868873596, -0.017982766032218933,
     -0.0852997675538063,  -0.032453566789627075, 0.0014507169835269451,
     0.03345133736729622,   0.048862338066101074,  0.006664620712399483,
    -0.06287197023630142,   -0.02109423652291298,

### Embed multiple texts

You can embed multiple texts for indexing with `embedDocuments`. The internals used for this method may (but do not have to) differ from embedding queries:

In [4]:
const text2 = "LangGraph is a library for building stateful, multi-actor applications with LLMs";

const vectors = await embeddings.embedDocuments([text, text2]);

console.log(vectors[0].slice(0, 100));
console.log(vectors[1].slice(0, 100));

[
    -0.02831101417541504,   0.022063178941607475,  -0.07454229146242142,
    0.006448323838412762,   0.001955120824277401, -0.017617391422390938,
       0.018649872392416,   -0.05262855067849159, 0.0006953597767278552,
  -0.0018249079585075378,   0.022437218576669693, 0.0036489504855126143,
   0.0018086736090481281,   0.016940006986260414, -0.007894322276115417,
    -0.04187627509236336,   0.039501357823610306,   0.06918870657682419,
   -0.006931832991540432,   0.049655742943286896,  0.021211417391896248,
   -0.029322246089577675,   -0.04546992480754852,  -0.01769082061946392,
    0.046703994274139404,    0.03127637133002281,  0.006355373188853264,
    0.014901148155331612,  -0.006893016863614321,  -0.05992589890956879,
   -0.009733330458402634,   0.015709295868873596, -0.017982766032218933,
     -0.0852997675538063,  -0.032453566789627075, 0.0014507169835269451,
     0.03345133736729622,   0.048862338066101074,  0.006664620712399483,
    -0.06287197023630142,   -0.02109423652291298,

## API reference

For detailed documentation of all `VertexAIEmbeddings` features and configurations head to the API reference: https://api.js.langchain.com/classes/langchain_google_vertexai.GoogleVertexAIEmbeddings.html